# Amazon SageMaker Autopilot Candidate Definition Notebook
The AutoML job **ccf-F1-score** generated this notebook automatically.
You can modify the candidate definitions and run the SageMaker Autopilot workflow using this notebook.

The target column is the **Class** column, which is one of the dataset's **31** columns. This is being handled as a problem with **BinaryClassification**. In addition, the dataset has **2** classes.
This notebook will create a binary classification model that **maximises** the trained models' "**F1**" quality metric (see https://en.wikipedia.org/wiki/Binary_classification).
For binary classification with a positive and negative class, the "**F1**" metric is used. It combines precision and recall and is advised in situations where there are more negative than positive examples.

The input dataset has been randomly divided into two parts for **training** and **validation** as part of the AutoML job. You can examine and alter the data transformation strategies suggested by Amazon SageMaker Autopilot using this notebook. The data transformation models can be interactively trained and used to transform the data. Finally, by jointly optimising the data transformations and machine learning algorithms, you may run a multiple algorithm hyperparameter optimisation (multi-algo HPO) project to help you find the optimum model for your dataset.



## Sagemaker Setup

Before you launch the SageMaker Autopilot jobs, we'll setup the environment for Amazon SageMaker
- Check environment & dependencies.
- Create a few helper objects/function to organize input/output data and SageMaker sessions.

**Minimal Environment Requirements**

- Jupyter: Tested on `JupyterLab 1.0.6`, `jupyter_core 4.5.0` and `IPython 6.4.0`
- Kernel: `conda_python3`
- Dependencies required
  - `sagemaker-python-sdk>=2.40.0`
    - Use `!pip install sagemaker==2.40.0` to download this dependency.
    - Kernel may need to be restarted after download.
- Expected Execution Role/permission
  - S3 access to the bucket that stores the notebook.

### Downloading Generated Modules
Download the generated data transformation modules and an SageMaker Autopilot helper module used by this notebook.
Those artifacts will be downloaded to **ccf-F1-score-artifacts** folder.

In [3]:
!mkdir -p ccf-F1-score-artifacts
!aws s3 sync s3://sagemaker-studio-057057355214-bh401jmpsm9/ccf-F1-score/sagemaker-automl-candidates/ccf-F1-score-pr-1-9fd7c14660d9409893495022065874b394e9a0aeb5644/generated_module ccf-F1-score-artifacts/generated_module --only-show-errors
!aws s3 sync s3://sagemaker-studio-057057355214-bh401jmpsm9/ccf-F1-score/sagemaker-automl-candidates/ccf-F1-score-pr-1-9fd7c14660d9409893495022065874b394e9a0aeb5644/notebooks/sagemaker_automl ccf-F1-score-artifacts/sagemaker_automl --only-show-errors

import sys
sys.path.append("ccf-F1-score-artifacts")

### SageMaker Autopilot Job and Amazon Simple Storage Service (Amazon S3) Configuration

The following configuration has been derived from the SageMaker Autopilot job. These items configure where this notebook will
look for generated candidates, and where input and output data is stored on Amazon S3.

In [4]:
from sagemaker_automl import uid, AutoMLLocalRunConfig

# Where the preprocessed data from the existing AutoML job is stored
BASE_AUTOML_JOB_NAME = 'ccf-F1-score'
BASE_AUTOML_JOB_CONFIG = {
    'automl_job_name': BASE_AUTOML_JOB_NAME,
    'automl_output_s3_base_path': 's3://sagemaker-studio-057057355214-bh401jmpsm9/ccf-F1-score',
    'data_transformer_image_repo_version': '2.4-1-cpu-py3',
    'algo_image_repo_versions': {'xgboost': '1.2-2-cpu-py3', 'linear-learner': 'latest', 'mlp': 'training-cpu'},
    'algo_inference_image_repo_versions': {'xgboost': '1.2-2-cpu-py3', 'linear-learner': 'latest', 'mlp': 'inference-cpu'}
}

# Path conventions of the output data storage path from the local AutoML job run of this notebook
LOCAL_AUTOML_JOB_NAME = 'ccf-F1-sco-notebook-run-{}'.format(uid())
LOCAL_AUTOML_JOB_CONFIG = {
    'local_automl_job_name': LOCAL_AUTOML_JOB_NAME,
    'local_automl_job_output_s3_base_path': 's3://sagemaker-studio-057057355214-bh401jmpsm9/ccf-F1-score/{}'.format(LOCAL_AUTOML_JOB_NAME),
    'data_processing_model_dir': 'data-processor-models',
    'data_processing_transformed_output_dir': 'transformed-data',
    'multi_algo_tuning_output_dir': 'multi-algo-tuning'
}

AUTOML_LOCAL_RUN_CONFIG = AutoMLLocalRunConfig(
    role='arn:aws:iam::057057355214:role/service-role/AmazonSageMaker-ExecutionRole-20211118T193999',
    base_automl_job_config=BASE_AUTOML_JOB_CONFIG,
    local_automl_job_config=LOCAL_AUTOML_JOB_CONFIG,
    security_config={'EnableInterContainerTrafficEncryption': False, 'VpcConfig': {}})

AUTOML_LOCAL_RUN_CONFIG.display()

This notebook is initialized to use the following configuration: 
        <table>
        <tr><th colspan=2>Name</th><th>Value</th></tr>
        <tr><th>General</th><th>Role</th><td>arn:aws:iam::057057355214:role/service-role/AmazonSageMaker-ExecutionRole-20211118T193999</td></tr>
        <tr><th rowspan=2>Base AutoML Job</th><th>Job Name</th><td>ccf-F1-score</td></tr>
        <tr><th>Base Output S3 Path</th><td>s3://sagemaker-studio-057057355214-bh401jmpsm9/ccf-F1-score</td></tr>
        <tr><th rowspan=5>Interactive Job</th><th>Job Name</th><td>ccf-F1-sco-notebook-run-03-01-19-34</td></tr>
        <tr><th>Base Output S3 Path</th><td>s3://sagemaker-studio-057057355214-bh401jmpsm9/ccf-F1-score/ccf-F1-sco-notebook-run-03-01-19-34</td></tr>
        <tr><th>Data Processing Trained Model Directory</th><td>s3://sagemaker-studio-057057355214-bh401jmpsm9/ccf-F1-score/ccf-F1-sco-notebook-run-03-01-19-34/data-processor-models</td></tr>
        <tr><th>Data Processing Transformed Output</th><td>s3://sagemaker-studio-057057355214-bh401jmpsm9/ccf-F1-score/ccf-F1-sco-notebook-run-03-01-19-34/transformed-data</td></tr>
        <tr><th>Algo Tuning Model Output Directory</th><td>s3://sagemaker-studio-057057355214-bh401jmpsm9/ccf-F1-score/ccf-F1-sco-notebook-run-03-01-19-34/multi-algo-tuning</td></tr>
        </table>
        

## Candidate Pipelines

The `AutoMLLocalRunner` keeps track of selected candidates and automates many of the steps needed to execute feature engineering and tuning steps.

In [5]:
from sagemaker_automl import AutoMLInteractiveRunner, AutoMLLocalCandidate

automl_interactive_runner = AutoMLInteractiveRunner(AUTOML_LOCAL_RUN_CONFIG)

### Generated Candidates

The SageMaker Autopilot Job has analyzed the dataset and has generated **7** machine learning
pipeline(s) that use **3** algorithm(s). Each pipeline contains a set of feature transformers and an
algorithm.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. The resource configuration: instance type & count
1. Select candidate pipeline definitions by cells
1. The linked data transformation script can be reviewed and updated. Please refer to the [README.md](./ccf-F1-score-artifacts/generated_module/README.md) for detailed customization instructions.
</div>

**[dpp0-xgboost](ccf-F1-score-artifacts/generated_module/candidate_data_processors/dpp0.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [6]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp0",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

2021-12-03 01:19:35,753 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:35,829 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 01:19:35,834 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:35,856 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 01:19:35,858 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:35,876 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.


**[dpp1-xgboost](ccf-F1-score-artifacts/generated_module/candidate_data_processors/dpp1.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [7]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp1",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

2021-12-03 01:19:40,616 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:40,631 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 01:19:40,634 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:40,662 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 01:19:40,664 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:40,719 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.


**[dpp2-linear-learner](ccf-F1-score-artifacts/generated_module/candidate_data_processors/dpp2.py)**: This data transformation strategy first transforms 'numeric' features using [combined RobustImputer and RobustMissingIndicator](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py) followed by [QuantileExtremeValuesTransformer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *linear-learner* model. Here is the definition:

In [8]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp2",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "linear-learner",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

2021-12-03 01:19:43,975 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:43,997 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 01:19:44,001 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:44,002 WARNING sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.
2021-12-03 01:19:44,024 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 01:19:44,026 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:44,027 WARNING sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.
2021-12-03 01:19:44,064 INFO sagemaker.image_uris: Ignoring unnecessary instance t

**[dpp3-xgboost](ccf-F1-score-artifacts/generated_module/candidate_data_processors/dpp3.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [9]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp3",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

2021-12-03 01:19:45,251 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:45,281 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 01:19:45,283 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:45,370 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 01:19:45,372 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:45,391 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.


**[dpp4-xgboost](ccf-F1-score-artifacts/generated_module/candidate_data_processors/dpp4.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [10]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp4",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

2021-12-03 01:19:46,267 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:46,290 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 01:19:46,292 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:46,312 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 01:19:46,319 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:46,347 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.


**[dpp5-xgboost](ccf-F1-score-artifacts/generated_module/candidate_data_processors/dpp5.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [11]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp5",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

2021-12-03 01:19:47,201 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:47,222 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 01:19:47,224 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:47,244 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 01:19:47,246 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:47,266 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.


**[dpp6-mlp](ccf-F1-score-artifacts/generated_module/candidate_data_processors/dpp6.py)**: This data transformation strategy transforms 'numeric' features using [RobustImputer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *mlp* model. Here is the definition:

In [12]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp6",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "mlp",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "candidate_specific_static_hyperparameters": {
            "num_categorical_features": '0',
        }
    }
})

2021-12-03 01:19:48,205 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:48,229 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 01:19:48,231 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:48,232 WARNING sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: training-cpu.
2021-12-03 01:19:48,254 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 01:19:48,256 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-12-03 01:19:48,257 WARNING sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: inference-cpu.
2021-12-03 01:19:48,378 INFO sagemaker.image_uris: Ignoring unnecessa

### Selected Candidates

You have selected the following candidates (please run the cell below and click on the feature transformer links for details):

In [13]:
automl_interactive_runner.display_candidates()

Candidate Name,Algorithm,Feature Transformer
dpp0-xgboost,xgboost,dpp0.py
dpp1-xgboost,xgboost,dpp1.py
dpp2-linear-learner,linear-learner,dpp2.py
dpp3-xgboost,xgboost,dpp3.py
dpp4-xgboost,xgboost,dpp4.py
dpp5-xgboost,xgboost,dpp5.py
dpp6-mlp,mlp,dpp6.py


The feature engineering pipeline consists of two SageMaker jobs:

1. Generated trainable data transformer Python modules like [dpp0.py](ccf-F1-score-artifacts/generated_module/candidate_data_processors/dpp0.py), which has been downloaded to the local file system
2. A **training** job to train the data transformers
3. A **batch transform** job to apply the trained transformation to the dataset to generate the algorithm compatible data

The transformers and its training pipeline are built using open sourced **[sagemaker-scikit-learn-container][]** and **[sagemaker-scikit-learn-extension][]**.

[sagemaker-scikit-learn-container]: https://github.com/aws/sagemaker-scikit-learn-container
[sagemaker-scikit-learn-extension]: https://github.com/aws/sagemaker-scikit-learn-extension

## Executing the Candidate Pipelines

Each candidate pipeline consists of two steps, feature transformation and algorithm training.
For efficiency first execute the feature transformation step which will generate a featurized dataset on S3
for each pipeline.

After each featurized dataset is prepared, execute a multi-algorithm tuning job that will run tuning jobs
in parallel for each pipeline. This tuning job will execute training jobs to find the best set of
hyper-parameters for each pipeline, as well as finding the overall best performing pipeline.

### Run Data Transformation Steps

Now you are ready to start execution all data transformation steps.  The cell below may take some time to finish,
feel free to go grab a cup of coffee. To expedite the process you can set the number of `parallel_jobs` to be up to 10.
Please check the account limits to increase the limits before increasing the number of jobs to run in parallel.

In [14]:
automl_interactive_runner.fit_data_transformers(parallel_jobs=2)

2021-12-03 01:25:10,859 INFO root: [Worker_0:dpp0-xgboost]Executing step: train_data_transformer
2021-12-03 01:25:10,861 INFO sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: latest.
2021-12-03 01:25:10,881 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 01:25:11,046 INFO sagemaker: Creating training-job with name: ccf-F1-sco-notebook-run-03-01-19-34-dpp0-train-03-01-25-09

2021-12-03 01:25:11 Starting - Starting the training job2021-12-03 01:25:12,863 INFO root: [Worker_1:dpp1-xgboost]Executing step: train_data_transformer
2021-12-03 01:25:12,864 INFO sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: latest.
2021-12-03 01:25:12,890 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 01:25:13,022 INFO sagemaker: Creating training-job with name: ccf-F1-sco-notebook-run-03-01-19-34-dpp1-train-03-01-25-09

2021-12-03 01:25:13 Starting - Starting the training job

### Multi Algorithm Hyperparameter Tuning

Now that the algorithm compatible transformed datasets are ready, you can start the multi-algorithm model tuning job
to find the best predictive model. The following algorithm training job configuration for each
algorithm is auto-generated by the AutoML Job as part of the recommendation.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. Hyperparameter ranges
2. Objective metrics
3. Recommended static algorithm hyperparameters.

Please refers to [Xgboost tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html) and [Linear learner tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner-tuning.html) for detailed explanations of the parameters.
</div>

The AutoML recommendation job has recommended the following hyperparameters, objectives and accuracy metrics for
the algorithm and problem type:

In [15]:
ALGORITHM_OBJECTIVE_METRICS = {
    'xgboost': 'validation:f1_binary',
    'linear-learner': 'validation:binary_f_beta',
    'mlp': 'validation:binary_f_beta',
}

STATIC_HYPERPARAMETERS = {
    'xgboost': {
        'objective': 'binary:logistic',
        'eval_metric': 'accuracy,f1_binary,auc',
        'scale_pos_weight': 577.8760162601626,
        'save_model_on_termination': 'true',
    },
    'linear-learner': {
        'predictor_type': 'binary_classifier',
        'loss': 'logistic',
        'mini_batch_size': 800,
        'binary_classifier_model_selection_criteria': 'loss_function',
        'num_models': 1,
        'positive_example_weight_mult': 577.8760162601626,
    },
    'mlp': {
        'problem_type': 'binary_classification',
        'positive_example_weight_mult': 577.8760162601626,
        'ml_application': 'mlp',
        'use_batchnorm': 'true',
        'activation': 'relu',
        'warmup_epochs': 10,
        'reporting_metrics': 'accuracy,binary_f_1,roc_auc',
        'eval_metric': 'binary_f_1',
    },
}

The following tunable hyperparameters search ranges are recommended for the Multi-Algo tuning job:

In [16]:
from sagemaker.parameter import CategoricalParameter, ContinuousParameter, IntegerParameter

ALGORITHM_TUNABLE_HYPERPARAMETER_RANGES = {
    'xgboost': {
        'num_round': IntegerParameter(64, 1024, scaling_type='Logarithmic'),
        'max_depth': IntegerParameter(2, 8, scaling_type='Logarithmic'),
        'eta': ContinuousParameter(1e-3, 1.0, scaling_type='Logarithmic'),
        'gamma': ContinuousParameter(1e-6, 64.0, scaling_type='Logarithmic'),
        'min_child_weight': ContinuousParameter(1e-6, 32.0, scaling_type='Logarithmic'),
        'subsample': ContinuousParameter(0.5, 1.0, scaling_type='Linear'),
        'colsample_bytree': ContinuousParameter(0.3, 1.0, scaling_type='Linear'),
        'lambda': ContinuousParameter(1e-6, 2.0, scaling_type='Logarithmic'),
        'alpha': ContinuousParameter(1e-6, 2.0, scaling_type='Logarithmic'),
    },
    'linear-learner': {
        'wd': ContinuousParameter(1e-7, 1.0, scaling_type='Logarithmic'),
        'l1': ContinuousParameter(1e-7, 1.0, scaling_type='Logarithmic'),
        'learning_rate': ContinuousParameter(1e-5, 1.0, scaling_type='Logarithmic'),
    },
    'mlp': {
        'mini_batch_size': IntegerParameter(128, 512, scaling_type='Linear'),
        'learning_rate': ContinuousParameter(1e-6, 1e-2, scaling_type='Logarithmic'),
        'weight_decay': ContinuousParameter(1e-12, 1e-2, scaling_type='Logarithmic'),
        'dropout_prob': ContinuousParameter(0.25, 0.5, scaling_type='Linear'),
        'embedding_size_factor': ContinuousParameter(0.65, 0.95, scaling_type='Linear'),
        'network_type': CategoricalParameter(['feedforward', 'widedeep']),
        'layers': CategoricalParameter(['256', '50, 25', '100, 50', '200, 100', '256, 128', '300, 150', '200, 100, 50']),
    },
}

#### Prepare Multi-Algorithm Tuner Input

To use the multi-algorithm HPO tuner, prepare some inputs and parameters. Prepare a dictionary whose key is the name of the trained pipeline candidates and the values are respectively:

1. Estimators for the recommended algorithm
2. Hyperparameters search ranges
3. Objective metrics

In [17]:
multi_algo_tuning_parameters = automl_interactive_runner.prepare_multi_algo_parameters(
    objective_metrics=ALGORITHM_OBJECTIVE_METRICS,
    static_hyperparameters=STATIC_HYPERPARAMETERS,
    hyperparameters_search_ranges=ALGORITHM_TUNABLE_HYPERPARAMETER_RANGES)

Below you prepare the inputs data to the multi-algo tuner:

In [18]:
multi_algo_tuning_inputs = automl_interactive_runner.prepare_multi_algo_inputs()

#### Create Multi-Algorithm Tuner

With the recommended Hyperparameter ranges and the transformed dataset, create a multi-algorithm model tuning job
that coordinates hyper parameter optimizations across the different possible algorithms and feature processing strategies.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. Tuner strategy: [Bayesian](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Bayesian_optimization), [Random Search](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Random_search)
2. Objective type: `Minimize`, `Maximize`, see [optimization](https://en.wikipedia.org/wiki/Mathematical_optimization)
3. Max Job size: the max number of training jobs HPO would be launching to run experiments. Note the default value is **250**
    which is the default of the managed flow.
4. Parallelism. Number of jobs that will be executed in parallel. Higher value will expedite the tuning process.
    Please check the account limits to increase the limits before increasing the number of jobs to run in parallel
5. Please use a different tuning job name if you re-run this cell after applied customizations.
</div>

In [19]:
from sagemaker.tuner import HyperparameterTuner

base_tuning_job_name = "{}-tuning".format(AUTOML_LOCAL_RUN_CONFIG.local_automl_job_name)

tuner = HyperparameterTuner.create(
    base_tuning_job_name=base_tuning_job_name,
    strategy='Bayesian',
    objective_type='Maximize',
    max_parallel_jobs=2,
    max_jobs=250,
    **multi_algo_tuning_parameters,
)

#### Run Multi-Algorithm Tuning

Now you are ready to start running the **Multi-Algo Tuning** job. After the job is finished, store the tuning job name which you use to select models in the next section.
The tuning process will take some time, please track the progress in the Amazon SageMaker Hyperparameter tuning jobs console.

In [ ]:
from IPython.display import display, Markdown

# Run tuning
tuner.fit(inputs=multi_algo_tuning_inputs, include_cls_metadata=None)
tuning_job_name = tuner.latest_tuning_job.name

display(
    Markdown(f"Tuning Job {tuning_job_name} started, please track the progress from [here](https://{AUTOML_LOCAL_RUN_CONFIG.region}.console.aws.amazon.com/sagemaker/home?region={AUTOML_LOCAL_RUN_CONFIG.region}#/hyper-tuning-jobs/{tuning_job_name})"))

# Wait for tuning job to finish
tuner.wait()

2021-12-03 02:06:09,330 INFO sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: latest.
2021-12-03 02:06:09,350 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 02:06:09,352 INFO sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: latest.
2021-12-03 02:06:09,373 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 02:06:09,374 INFO sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: latest.
2021-12-03 02:06:09,394 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 02:06:09,396 INFO sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: latest.
2021-12-03 02:06:09,415 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-12-03 02:06:09,417 INFO sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: latest.
2021-12-03 02:06:09,437

## Model Selection and Deployment

This section guides you through the model selection process. Afterward, you construct an inference pipeline
on Amazon SageMaker to host the best candidate.

Because you executed the feature transformation and algorithm training in two separate steps, you now need to manually
link each trained model with the feature transformer that it is associated with. When running a regular Amazon
SageMaker Autopilot job, this will automatically be done for you.

### Tuning Job Result Overview

The performance of each candidate pipeline can be viewed as a Pandas dataframe. For more interactive usage please
refers to [model tuning monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-monitor.html).

In [24]:
from pprint import pprint
from sagemaker.analytics import HyperparameterTuningJobAnalytics

SAGEMAKER_SESSION = AUTOML_LOCAL_RUN_CONFIG.sagemaker_session
SAGEMAKER_ROLE = AUTOML_LOCAL_RUN_CONFIG.role

tuner_analytics = HyperparameterTuningJobAnalytics(
    tuner.latest_tuning_job.name, sagemaker_session=SAGEMAKER_SESSION)

df_tuning_job_analytics = tuner_analytics.dataframe()

# Sort the tuning job analytics by the final metrics value
df_tuning_job_analytics.sort_values(
    by=['FinalObjectiveValue'],
    inplace=True,
    ascending=False if tuner.objective_type == "Maximize" else True)

# Show detailed analytics for the top 20 models
df_tuning_job_analytics.head(20)

,alpha,colsample_bytree,eta,gamma,lambda,max_depth,min_child_weight,num_round,subsample,TrainingJobName,...,TrainingJobDefinitionName,dropout_prob,embedding_size_factor,layers,learning_rate,mini_batch_size,network_type,weight_decay,l1,wd
30,0.000003,0.945699,0.319618,0.000005,0.510349,3.0,0.359117,573.0,0.603489,ccf-F1-sco-notebook--211203-0206-220-6ea75adc,...,dpp3-xgboost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,0.000158,0.532783,0.439610,0.000225,0.004294,5.0,9.056962,795.0,0.912518,ccf-F1-sco-notebook--211203-0206-204-d8ecc56f,...,dpp3-xgboost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,0.026534,0.730017,0.177250,0.000002,2.000000,5.0,0.000041,407.0,0.616748,ccf-F1-sco-notebook--211203-0206-208-835ef8b4,...,dpp3-xgboost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,0.026814,0.624249,0.265388,0.739151,0.000104,5.0,13.600228,1024.0,0.837048,ccf-F1-sco-notebook--211203-0206-236-7bc617b0,...,dpp3-xgboost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,0.000005,0.505097,0.278809,0.000017,0.000612,5.0,4.788247,775.0,0.917277,ccf-F1-sco-notebook--211203-0206-128-47f9a4db,...,dpp3-xgboost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249,0.000121,0.609035,0.086090,0.000005,0.078096,4.0,0.000004,455.0,0.591670,ccf-F1-sco-notebook--211203-0206-001-158129f8,...,dpp3-xgboost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,0.020637,0.457469,0.219402,0.000001,0.000112,2.0,3.778362,1024.0,0.777983,ccf-F1-sco-notebook--211203-0206-190-d06d9b0d,...,dpp3-xgboost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,0.000012,0.563174,0.109192,0.000001,0.946195,8.0,3.914601,644.0,0.644743,ccf-F1-sco-notebook--211203-0206-127-62e8e126,...,dpp3-xgboost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,0.021682,0.790943,0.219503,0.000019,1.629311,7.0,0.035587,604.0,0.597469,ccf-F1-sco-notebook--211203-0206-114-6bc46622,...,dpp3-xgboost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,0.000427,0.668270,0.615661,0.000419,1.204446,3.0,0.000028,994.0,0.524313,ccf-F1-sco-notebook--211203-0206-178-31360477,...,dpp3-xgboost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The best training job can be selected as below:

<div class="alert alert-info"> 💡 <strong>Tips: </strong>
You could select alternative job by using the value from `TrainingJobName` column above and assign to `best_training_job` below
</div>

In [25]:
attached_tuner = HyperparameterTuner.attach(tuner.latest_tuning_job.name, sagemaker_session=SAGEMAKER_SESSION)
best_training_job = attached_tuner.best_training_job()

print("Best Multi Algorithm HPO training job name is {}".format(best_training_job))

Best Multi Algorithm HPO training job name is ccf-F1-sco-notebook--211203-0206-220-6ea75adc


### Linking Best Training Job with Feature Pipelines

Finally, deploy the best training job to Amazon SageMaker along with its companion feature engineering models.
At the end of the section, you get an endpoint that's ready to serve online inference or start batch transform jobs!

Deploy a [PipelineModel](https://sagemaker.readthedocs.io/en/stable/pipeline.html) that has multiple containers of the following:

1. Data Transformation Container: a container built from the model we selected and trained during the data transformer sections
2. Algorithm Container: a container built from the trained model we selected above from the best HPO training job.
3. Inverse Label Transformer Container: a container that converts numerical intermediate prediction value back to non-numerical label value.

Get both best data transformation model and algorithm model from best training job and create an pipeline model:

In [26]:
from sagemaker.estimator import Estimator
from sagemaker import PipelineModel
from sagemaker_automl import select_inference_output

# Get a data transformation model from chosen candidate
best_candidate = automl_interactive_runner.choose_candidate(df_tuning_job_analytics, best_training_job)
best_data_transformer_model = best_candidate.get_data_transformer_model(role=SAGEMAKER_ROLE, sagemaker_session=SAGEMAKER_SESSION)

# Our first data transformation container will always return recordio-protobuf format
best_data_transformer_model.env["SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT"] = 'application/x-recordio-protobuf'
# Add environment variable for sparse encoding
if best_candidate.data_transformer_step.sparse_encoding:
    best_data_transformer_model.env["AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF"] = '1'

# Get a algo model from chosen training job of the candidate
algo_estimator = Estimator.attach(best_training_job)
best_algo_model = algo_estimator.create_model(**best_candidate.algo_step.get_inference_container_config())

# Final pipeline model is composed of data transformation models and algo model and an
# inverse label transform model if we need to transform the intermediates back to non-numerical value
model_containers = [best_data_transformer_model, best_algo_model]
if best_candidate.transforms_label:
    model_containers.append(best_candidate.get_data_transformer_model(
        transform_mode="inverse-label-transform",
        role=SAGEMAKER_ROLE,
        sagemaker_session=SAGEMAKER_SESSION))

# This model can emit response ['predicted_label', 'probability', 'labels', 'probabilities']. To enable the model to emit one or more
# of the response content, pass the keys to `output_key` keyword argument in the select_inference_output method.

model_containers = select_inference_output("BinaryClassification", model_containers, output_keys=['predicted_label'])


pipeline_model = PipelineModel(
    name="AutoML-{}".format(AUTOML_LOCAL_RUN_CONFIG.local_automl_job_name),
    role=SAGEMAKER_ROLE,
    models=model_containers,
    vpc_config=AUTOML_LOCAL_RUN_CONFIG.vpc_config)

2021-12-04 15:01:54,858 INFO root: Chosen Data Processing pipeline candidate name is dpp3-xgboost

2021-12-03 01:39:32 Starting - Preparing the instances for training
2021-12-03 01:39:32 Downloading - Downloading input data
2021-12-03 01:39:32 Training - Training image download completed. Training in progress.
2021-12-03 01:39:32 Uploading - Uploading generated training model
2021-12-03 01:39:32 Completed - Training job completed

2021-12-03 14:35:02 Starting - Preparing the instances for training
2021-12-03 14:35:02 Downloading - Downloading input data
2021-12-03 14:35:02 Training - Training image download completed. Training in progress.
2021-12-03 14:35:02 Uploading - Uploading generated training model
2021-12-03 14:35:02 Completed - Training job completed

2021-12-03 01:39:32 Starting - Preparing the instances for training
2021-12-03 01:39:32 Downloading - Downloading input data
2021-12-03 01:39:32 Training - Training image download completed. Training in progress.
2021-12-03 01:39

### Deploying Best Pipeline

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. You can customize the initial instance count and instance type used to deploy this model.
2. Endpoint name can be changed to avoid conflict with existing endpoints.

</div>

Finally, deploy the model to SageMaker to make it functional.

In [1]:
pipeline_model.deploy(initial_instance_count=1,
                      instance_type='ml.t2.medium', # Changed from ml.m5.2xlarge to allow for endpoint creation in AWS free tier
                      endpoint_name=pipeline_model.name,
                      wait=True)